In [1]:
import numpy as np
import random
from collections import defaultdict
import ast
import json
import psutil
import pynvml
import matplotlib.pyplot as plt

from environment_ma_reward_distance_dynamic import Env

class ProblemSolver:
    def __init__(self, num_actions, env, alpha, gamma, epsilon):
        self.env = env
        self.num_actions = num_actions
        self.learning_rate = alpha
        self.discount_factor = gamma
        self.epsilon = epsilon
        self.q_tables = [defaultdict(lambda: [0.0] * num_actions) for _ in range(env.num_agents)]

    @staticmethod
    def arg_max(state_action):
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

    def choose_action(self, agent_idx, state):
        state = tuple(state)
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.num_actions)
        else:
            state_action = self.q_tables[agent_idx][state]
            action = self.arg_max(state_action)
        return action

    def learn(self, agent_idx, state, action, reward, next_state):
        state = tuple(state)
        next_state = tuple(next_state)
        current_q = self.q_tables[agent_idx][state][action]
        max_next_q = max(self.q_tables[agent_idx][next_state])
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_next_q - current_q)
        self.q_tables[agent_idx][state][action] = new_q


class Case:
    added_states = set()  # Store states already added to the case base

    def __init__(self, problem, solution, trust_value=0.5):
        self.problem = ast.literal_eval(problem) if isinstance(problem, str) else problem
        self.solution = solution
        self.trust_value = trust_value
    
    @staticmethod
    def sim_q(state1, state2):
        state1 = np.atleast_1d(state1)
        state2 = np.atleast_1d(state2)
        CNDMaxDist = 6
        v = state1.size
        DistQ = np.sum([Case.dist_q(Objic, Objip) for Objic, Objip in zip(state1, state2)])
        similarity = (CNDMaxDist * v - DistQ) / (CNDMaxDist * v)
        return similarity

    @staticmethod
    def dist_q(X1, X2):
        return np.min(np.abs(X1 - X2))

    @staticmethod
    def retrieve(state, case_base, threshold=0.1):
        state = ast.literal_eval(state) if isinstance(state, str) else state
        for case in case_base:
            if state == case.problem: 
                return case

    @staticmethod
    def reuse(c, temporary_case_base):
        temporary_case_base.append(c)

    @staticmethod
    def revise(case_base, temporary_case_base, successful_episodes):
        for case in case_base:
            if any((case.problem, case.solution) == (temp_case.problem, temp_case.solution) for temp_case in temporary_case_base):
                if successful_episodes:
                    case.trust_value += 0.1
                else:
                    case.trust_value -= 0.1
            else:
                if successful_episodes:
                    case.trust_value -= 0.1
            
            case.trust_value = max(0, min(case.trust_value, 1))
            print(f"case content after REVISE, problem: {case.problem}, solution: {case.solution}, tv: {case.trust_value}")

    @staticmethod
    def retain(case_base, temporary_case_base, successful_episodes, threshold=0.29):
        if successful_episodes:
            for temp_case in reversed(temporary_case_base):
                state = tuple(np.atleast_1d(temp_case.problem))
                if not any(tuple(np.atleast_1d(case.problem)) == state for case in case_base):
                    case_base.append(temp_case)
                    # Case.added_states.add(state)
                    print(f"Episode succeeded, case {temp_case.problem} is empty. Temporary case base stored to the case base: {temp_case.problem, temp_case.solution, temp_case.trust_value,}")         
                else:
                    existing_case = next((case for case in case_base if tuple(np.atleast_1d(case.problem)) == state), None)
                    if existing_case and existing_case.trust_value < temp_case.trust_value:
                        existing_case.solution = temp_case.solution
                        existing_case.trust_value = max(0, temp_case.trust_value)
                        print(f"Episode succeeded, similar case found. Updated case base with higher trust value: {existing_case.problem, existing_case.solution, existing_case.trust_value}")
                    else:
                        print(f"Episode succeeded, case {temp_case.problem} is not empty. Temporary case base that not stored to the case base: {temp_case.problem, temp_case.solution, temp_case.trust_value,}")    
        else:
            print(f"Episode not succeeded, temporary case base not stored to the case base")
        
        case_base[:] = [case for case in case_base if case.trust_value >= threshold]

        for case in case_base:
            print(f"cases content after RETAIN, problem: {case.problem}, solution: {case.solution}, tv: {case.trust_value}")

        return case_base


class QCBRL:
    def __init__(self, num_actions, env, episodes, max_steps, alpha, gamma, epsilon, epsilon_decay, epsilon_min, render):
        self.num_actions = num_actions
        self.env = env
        self.episodes = episodes
        self.max_steps = max_steps
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.render = render
        self.epsilon_decay = epsilon_decay  
        self.epsilon_min = epsilon_min  

        self.problem_solvers = [ProblemSolver(num_actions, self.env, alpha, gamma, epsilon) for _ in range(self.env.num_agents)]
        self.case_base = []  # Centralized case base
        self.temporary_case_bases = [[] for _ in range(self.env.num_agents)]
        self.successful_episodes = [0] * self.env.num_agents
        self.rewards_per_episode = [[] for _ in range(self.env.num_agents)]  
        self.total_successful_episodes = 0 
        self.action_type = 0

    def run(self):
        rewards = []
        memory_usage = []
        gpu_memory_usage = []
        num_successful_episodes = 0
        total_steps_list = []
        success_steps = []

        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)

        for episode in range(episodes):
            states = self.env.reset()
            episode_reward = [0] * self.env.num_agents
            total_steps = 0 
            self.temporary_case_bases = [[] for _ in range(self.env.num_agents)]
            success_count = [0] * self.env.num_agents
            dones = [False] * self.env.num_agents
            win_states = [False] * self.env.num_agents
            successful_episodes = False

            while not(all(dones)):
                
                actions = []
                for agent_idx in range(self.env.num_agents):
                    state = states[agent_idx]
                    # print(f"state before take action: {state}")
                    action = self.take_action(agent_idx, state)
                    actions.append(action)

                # print(f"actions pass to the environment")
                next_states, rewards, dones = self.env.step(actions)

                win_states = []
                for agent_idx in range(self.env.num_agents):
                    state = states[agent_idx]
                    action = actions[agent_idx]
                    reward = rewards[agent_idx]
                    next_state = next_states[agent_idx]

                    physical_state = tuple(state[0])
                    win_state = state[1]
                    comm_state = state[2]

                    physical_next_state = tuple(next_state[0])
                    win_next_state = next_state[1]
                    comm_next_state = next_state[2]

                    physical_action = action[0]
                    comm_action = action[1]

                    # print(f"state agent {agent_idx} before update: {physical_state}")
                    # print(f"win state agent {agent_idx} before update: {win_next_state}")
                    # print(f"action agent {agent_idx} before update: {physical_action}")
                    # print(f"reward agent {agent_idx} before update: {reward}")
                    # print(f"next state agent {agent_idx} before update: {physical_next_state}")

                    c = Case(physical_state, physical_action)
                    Case.reuse(c, self.temporary_case_bases[agent_idx])

                    if self.action_type == 0:
                        print(f"action type: problem solver, agent learned")
                        self.problem_solvers[agent_idx].learn(agent_idx, physical_state, physical_action, reward, physical_next_state)

                    if (win_next_state): 
                        success_count[agent_idx] += 1
                        print(f"agent{agent_idx} hit !!!!!")
                    else:
                        print(f"agent{agent_idx} not hit !!!!!")

                    episode_reward[agent_idx] += reward
                    win_states.append(win_next_state)  

                states = next_states
                total_steps += 1

                self.env.render()
                
            if self.env.win_flag:
                self.total_successful_episodes += 1
                success_steps.append(total_steps)
                successful_episodes = True
                

            print(f"Episode: {episode}, Total Steps: {total_steps}, Total Rewards: {episode_reward}, Status Episode: {successful_episodes}")
            print("--------------------")

            for agent_idx in range(self.env.num_agents):
                print(f"win next of {agent_idx} state before update the case base: {win_states}")
                self.rewards_per_episode[agent_idx].append(episode_reward[agent_idx])
                
                Case.revise(self.case_base, self.temporary_case_bases[agent_idx], win_states[agent_idx])
                self.case_base = Case.retain(self.case_base, self.temporary_case_bases[agent_idx], win_states[agent_idx])
               
                
            self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)
            
            memory_usage.append(psutil.virtual_memory().percent)
            gpu_memory_usage.append(pynvml.nvmlDeviceGetMemoryInfo(handle).used / 1024**2)

        self.save_case_base_temporary()  # Save temporary case base after training
        self.save_case_base()  # Save case base after training

        success_rate = self.total_successful_episodes / episodes * 100


        return self.rewards_per_episode, success_rate, memory_usage, gpu_memory_usage, success_steps

    def take_action(self, agent_idx, state):
        # print(f"state detected in take action function: {state}")
        physical_state = tuple(state[0])
        win_state = state[1]
        comm_state = state[2]

        similar_solution = Case.retrieve(physical_state, self.case_base)
        if similar_solution is not None:
            physical_action = similar_solution.solution
            self.action_type = 1
            print(f"Problem detected as a similiar soulution in case base: {similar_solution.problem}")
            print(f"Action for Agent {agent_idx} from case base: {physical_action}")
            print(f"Trust value detected as a similiar solution in case base: {similar_solution.trust_value}")
        else:
            physical_action = self.problem_solvers[agent_idx].choose_action(agent_idx, physical_state)
            self.action_type = 0
            print(f"Action for Agent {agent_idx} from problem solver: {physical_action}")

        if self.env.is_agent_silent:
            communication_action = None
        else:
            if similar_solution is not None:
                communication_action = f"Message send from agent {agent_idx}"
            else:
                communication_action = None
        
        # print(f"physical action returned from the take action: {physical_action}")
        # print(f"comm action returned from the take action: {communication_action}")

        return (physical_action, communication_action)
        
    
    def save_case_base_temporary(self):
        for agent_idx in range(self.env.num_agents):
            filename = f"cases/case_base_temporary_agent_{agent_idx}.json"
            case_base_data = [{"problem": case.problem.tolist() if isinstance(case.problem, np.ndarray) else case.problem, 
                            "solution": int(case.solution), 
                            "trust_value": int(case.trust_value)} for case in self.temporary_case_bases[agent_idx]]
            with open(filename, 'w') as file:
                json.dump(case_base_data, file)
            print(f"Temporary case base for Agent {agent_idx} saved successfully.")

    def save_case_base(self):
        filename = f"cases/case_base_centralized.json"
        case_base_data = [{"problem": case.problem.tolist() if isinstance(case.problem, np.ndarray) else case.problem, 
                        "solution": int(case.solution), 
                        "trust_value": int(case.trust_value)} for case in self.case_base]
        with open(filename, 'w') as file:
            json.dump(case_base_data, file)
        print(f"Centralized case base saved successfully.")
        
    def load_case_base(self):
        filename = f"cases/case_base_centralized.json"
        try:
            with open(filename, 'r') as file:
                case_base_data = json.load(file)
                self.case_base = [Case(np.array(case["problem"]), case["solution"], case["trust_value"]) for case in case_base_data]
                print(f"Centralized case base loaded successfully.")
        except FileNotFoundError:
            print(f"Centralized case base file not found. Starting with an empty case base.")

    def display_success_rate(self, success_rate):
        print(f"Success rate: {success_rate}%")


    def plot_rewards(self, rewards):
        for agent_idx in range(self.env.num_agents):
            plt.plot([reward for reward in rewards[agent_idx]], label=f'Agent {agent_idx}')
        plt.xlabel('Episode')
        plt.ylabel('Total Reward')
        plt.title('Rewards over Episodes')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_total_steps(self, total_steps_list):
        plt.plot(total_steps_list)
        plt.xlabel('Episode')
        plt.ylabel('Total Steps')
        plt.title('Total Steps for Successful Episodes over Episodes')
        plt.grid(True)
        plt.show()

    def plot_resources(self, memory_usage, gpu_memory_usage):
        plt.plot(memory_usage, label='Memory (%)')
        plt.plot(gpu_memory_usage, label='GPU Memory (MB)')
        plt.xlabel('Episode')
        plt.ylabel('Resource Usage')
        plt.title('Resource Usage over Episodes')
        plt.legend()
        plt.grid(True)
        plt.show()


if __name__ == "__main__":
    num_agents = 2
    num_obstacles = 3
    obstacles_random_steps = 25
    is_agent_silent = False
    episodes=49
    max_steps=1000
    alpha=0.1
    gamma=0.9
    epsilon=0.2
    epsilon_decay = 0.995  
    epsilon_min = 0.01  
    render = True

    env = Env(num_agents=num_agents, num_obstacles=num_obstacles, obstacles_random_steps = obstacles_random_steps, is_agent_silent=is_agent_silent)
    
    num_actions = len(env.action_space)
    
    agent = QCBRL(num_actions, env, episodes, max_steps, alpha, gamma, epsilon, epsilon_decay, epsilon_min, render)
    rewards, success_rate, memory_usage, gpu_memory_usage, total_step_list = agent.run()

    agent.display_success_rate(success_rate)
    agent.plot_rewards(rewards)
    agent.plot_total_steps(total_step_list)
    agent.plot_resources(memory_usage, gpu_memory_usage)


Action for Agent 0 from problem solver: 0
Action for Agent 1 from problem solver: 2
agent 0 is ongoing!
win status agent 0 = False
agent 1 hit the obstacle!
win status agent 1 = False
wins all agent situation in the environment: [False, False]
action type: problem solver, agent learned
agent0 not hit !!!!!
action type: problem solver, agent learned
agent1 not hit !!!!!
Action for Agent 0 from problem solver: 3
Action for Agent 1 from problem solver: 0
agent 0 is ongoing!
win status agent 0 = False
agent 1 is locked. Done status: True, win status: False
wins all agent situation in the environment: [False, False]
action type: problem solver, agent learned
agent0 not hit !!!!!
action type: problem solver, agent learned
agent1 not hit !!!!!
Action for Agent 0 from problem solver: 4
Action for Agent 1 from problem solver: 2
agent 0 is ongoing!
win status agent 0 = False
agent 1 is locked. Done status: True, win status: False
wins all agent situation in the environment: [False, False]
action

KeyboardInterrupt: 